# Data Preprocessing

cleaning  
reduction  
integration  

for seoul housing data.

## Functions

In [1]:
from sklearn import preprocessing
import pandas as pd
import xlrd

## Merging

Seoul Housing yearly data

In [29]:
## Merging seoul apartment housing data from 201505 to 201904

all_files = []
for i in range(2015,2020):
    all_files.append('seoul_apart_'+str(i)+'.csv') # prepare csv files
all_seoul_apart = pd.concat([pd.read_csv(f) for f in all_files]) # concatnate apart data

In [85]:
all_seoul_apart.to_csv('seoul_apart.csv') # save.

In [39]:
all_seoul_apart.head(5)

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201411,1~10,"59,500",4,1988,언주로
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201501,11~20,"60,000",5,1988,언주로
2,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201503,1~10,"65,000",5,1988,언주로
3,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201503,1~10,"62,500",2,1988,언주로
4,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201407,21~31,"58,800",4,1987,언주로


In [51]:
areacode = pd.read_csv('https://goo.gl/tM6r3v', sep='\t', dtype={'법정동코드':str}) # url for 법정동코드 

In [36]:
areacode = areacode[area_code['폐지여부'] == '존재'] # drop unused code

,법정동코드,법정동명,폐지여부
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재


In [38]:
seoul_areacode = area_code[area_code['법정동코드'].str.contains('11\d{8}')] # extract seoul region
seoul_areacode.to_csv('seoul_areacode.csv') # save

In [62]:
# original data doesn't have area code, only have name of area
# matching area name to code

def merging_areacode(sac, asa):
    sacd = sac.drop(columns=['폐지여부'], axis=1)
    itg = pd.merge(sacd, asa.rename(columns={'시군구':'법정동명'}), on='법정동명', how='right')
    return itg
        
seoul_apart_areacode = merging_areacode(seoul_areacode, all_seoul_apart)

In [66]:
seoul_apart_areacode.to_csv('seoul_apart_areacode.csv', index=False) # save.

## cleaning seoul housing data

there are too many columns that we don't need  
let's cut it down first.

In [2]:
seoul_housing_raw = pd.read_csv('seoul_apart_areacode.csv') # load csv file.

In [3]:
seoul_housing_raw.head(3)

,법정동코드,법정동명,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명
0,1111010100,서울특별시 종로구 청운동,56-45,56,45,청운현대,84.51,201405,1~10,"62,500",3,2000,자하문로33길
1,1111010100,서울특별시 종로구 청운동,56-45,56,45,청운현대,129.76,201503,11~20,"89,000",1,2000,자하문로33길
2,1111010100,서울특별시 종로구 청운동,56-45,56,45,청운현대,84.51,201507,11~20,"65,000",5,2000,자하문로33길


In [4]:
# drop columns which are useless for modelling
seoul_housing_droped = seoul_housing_raw.drop(columns=['번지', '본번', '부번',
                                                       '계약일', '도로명'])
seoul_housing_droped.head(5)

,법정동코드,법정동명,단지명,전용면적(㎡),계약년월,거래금액(만원),층,건축년도
0,1111010100,서울특별시 종로구 청운동,청운현대,84.51,201405,"62,500",3,2000
1,1111010100,서울특별시 종로구 청운동,청운현대,129.76,201503,"89,000",1,2000
2,1111010100,서울특별시 종로구 청운동,청운현대,84.51,201507,"65,000",5,2000
3,1111010100,서울특별시 종로구 청운동,청운현대,129.76,201612,"87,000",1,2000
4,1111010100,서울특별시 종로구 청운동,청운현대,84.51,201707,"79,000",5,2000


In [5]:
# load file that we made manually
seoul_other_features = pd.read_csv('seoul_other_features.csv')

In [37]:
# integrating housing data with features

def integration_housing_with_features(shd, sof):
    sof_group = sof.groupby(['법정동 코드'])
    gsof = pd.DataFrame({'랜드마크' : sof_group['랜드마크'].mean(),
                         '영화관' : sof_group['영화관'].mean(),
                         '쇼핑센터' : sof_group['쇼핑센터'].mean(),
                         '대형마트' : sof_group['대형마트'].mean(),
                         '백화점' : sof_group['백화점'].mean(),
                         '지하철' : sof_group['지하철'].mean(),
                         '차선수' : sof_group['차선수'].mean(),
                         '순환도로' : sof_group['순환도로'].mean(),
                         '한강' : sof_group['한강'].mean(),
                         '공원' : sof_group['공원'].mean(),
                         '산' : sof_group['산'].mean(),
                         '학교' : sof_group['학교'].mean(),
                         '교육특구' : sof_group['교육특구'].mean(),
                         '대학병원' : sof_group['대학병원'].mean()}).reset_index()
    itg = pd.merge(shd, gsof.rename(columns={'법정동 코드':'법정동코드'}), on='법정동코드', how='left')
    return itg
seoul_housing_integrated = integration_housing_with_features(seoul_housing_droped, seoul_other_features)
#print(seoul_other_features)

In [38]:
# drop NA rows
seoul_housing_integrated = seoul_housing_integrated.dropna()

In [39]:
shi=seoul_housing_integrated

In [13]:
# drop useless columns
seoul_housing_integrated = seoul_housing_integrated.drop(columns=['단지명', '법정동명'])

In [10]:
seoul_housing_integrated.head(1)

,법정동코드,법정동명,단지명,전용면적(㎡),계약년월,거래금액(만원),층,건축년도,랜드마크,영화관,...,백화점,지하철,차선수,순환도로,한강,공원,산,학교,교육특구,대학병원
9780,1117010100,서울특별시 용산구 후암동,(41-1),83.04,201405,"45,700",4,2012,2.0,0.0,...,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0


In [16]:
# save
seoul_housing_integrated.to_csv('seoul_housing_integrated.csv', index=False)

## Exploring Data

In [40]:
shi.head(1)

,법정동코드,법정동명,단지명,전용면적(㎡),계약년월,거래금액(만원),층,건축년도,랜드마크,영화관,...,백화점,지하철,차선수,순환도로,한강,공원,산,학교,교육특구,대학병원
9780,1117010100,서울특별시 용산구 후암동,(41-1),83.04,201405,"45,700",4,2012,2.0,0.0,...,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0


In [26]:
# do some explore for the dataset.
# grouping by name of the areaand exploring 

gr = shi.groupby(by=['법정동명'])
sh = pd.DataFrame(data={'거래금액(최대)':gr['거래금액(만원)'].max(),
                        '거래금액(최소)':gr['거래금액(만원)'].min()})

In [56]:
# load price

price = pd.read_csv('temp.csv')
k = shi.reset_index(drop=True)
k['거래금액']=price

In [54]:
k['거래금액']=shi['거래금액(만원)']

In [58]:
k['거래금액'] = k['거래금액'].astype(int)
k['면적당금액'] = k['거래금액']/(k['전용면적(㎡)'])

In [60]:
# grouping for various price types.

gr = k.groupby(by=['법정동명'])
sh = pd.DataFrame(data={'거래금액(최대)':gr['거래금액'].max(),
                        '거래금액(최소)':gr['거래금액'].min(),
                        '거래금액(평균)':gr['거래금액'].mean(),
                        '면적당금액(최대)':gr['면적당금액'].max(),
                        '면적당금액(최소)':gr['면적당금액'].min(),
                        '면적당금액(평균)':gr['면적당금액'].mean()})

In [61]:
sh.sort_values(by='면적당금액(최대)', ascending=False) # the most expensive apartment by the area size of the house

,거래금액(최대),거래금액(최소),거래금액(평균),면적당금액(최대),면적당금액(최소),면적당금액(평균)
법정동명,,,,,,
서울특별시 강남구 개포동,260000,22300,90998.107307,4935.370153,441.640379,1708.705793
서울특별시 강동구 둔촌동,170000,19500,68579.165317,3933.910307,296.101949,868.399568
서울특별시 서초구 반포동,497000,35000,164066.954248,3648.346475,347.532519,1554.372303
서울특별시 강남구 청담동,640000,18790,130526.017664,3516.798001,476.089716,1237.786831
서울특별시 용산구 한남동,840000,13270,189648.574383,3432.087567,247.402276,1247.390538
서울특별시 서초구 잠원동,345000,13500,108337.477885,3312.945973,443.636752,1277.419457
서울특별시 강남구 대치동,390000,18000,138579.607871,3250.541757,358.900269,1413.042004
서울특별시 송파구 잠실동,387000,35900,118025.767154,3247.832370,491.976991,1411.462172
서울특별시 강남구 압구정동,525000,78000,196063.239602,3201.280512,916.478279,1661.772340


In [62]:
print(sh[sh['거래금액(최대)']==sh['거래금액(최대)'].max()]) # the most expensive apartment's area in seoul

               거래금액(최대)  거래금액(최소)       거래금액(평균)    면적당금액(최대)   면적당금액(최소)  \
법정동명                                                                        
서울특별시 용산구 한남동    840000     13270  189648.574383  3432.087567  247.402276   

                 면적당금액(평균)  
법정동명                        
서울특별시 용산구 한남동  1247.390538  


In [63]:
k[k['거래금액']==k['거래금액'].max()] # the most expensive apartment

,법정동코드,법정동명,단지명,전용면적(㎡),계약년월,거래금액(만원),층,건축년도,랜드마크,영화관,...,차선수,순환도로,한강,공원,산,학교,교육특구,대학병원,거래금액,면적당금액
9273,1117013100,서울특별시 용산구 한남동,한남더힐,244.749,201901,"840,000",3,2011,1.0,0.0,...,8.0,1.0,2.0,3.0,1.0,3.0,0.0,1.0,840000,3432.087567


In [99]:
k.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 273838 entries, 9780 to 480383
Data columns (total 24 columns):
법정동코드       273838 non-null int64
법정동명        273838 non-null object
단지명         273838 non-null object
전용면적(㎡)     273838 non-null float64
계약년월        273838 non-null int64
거래금액(만원)    273838 non-null object
층           273838 non-null int64
건축년도        273838 non-null int64
랜드마크        273838 non-null float64
영화관         273838 non-null float64
쇼핑센터        273838 non-null float64
대형마트        273838 non-null float64
백화점         273838 non-null float64
지하철         273838 non-null float64
차선수         273838 non-null float64
순환도로        273838 non-null float64
한강          273838 non-null float64
공원          273838 non-null float64
산           273838 non-null float64
학교          273838 non-null float64
교육특구        273838 non-null float64
대학병원        273838 non-null float64
거래금액        273838 non-null int64
평당금액        273838 non-null float64
dtypes: float64(16), int64(5), obje